In [1]:
import pandas as pd
from pandas import read_excel
import pickle
trainData=read_excel('spam.xls') # DATA FRAME -> COL, ROWS
dispdata=read_excel('spam.xls')
# dispdata=trainData
#pre-processing
from bs4 import BeautifulSoup
import re

def html_tag(phrase):
    http_remove = re.sub(r"http\S+", "",phrase)
    html_remove = BeautifulSoup(http_remove, 'lxml').get_text()
    return html_remove

#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

from tqdm import tqdm
processed_review = []
for i in tqdm(trainData["Content"].values):
    sentance = html_tag(i)
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance)
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = " ".join(i.lower() for i in sentance.split() if i.lower() not in stopwords)
    processed_review.append(sentance)
    
trainData["Cleantext"] = processed_review

from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['Cleantext'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_vectors, trainData['Label'], test_size = 0.20, random_state = 0)

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 


model = pickle.load(open("model_svm.sav", 'rb'))
y_score = model.predict(X_test)
from sklearn.metrics import accuracy_score
svnacc=accuracy_score(y_test, y_score)
svnacc=round(svnacc*100,2)
print("Svm :"+ str(svnacc))
plot_confusion_matrix(model, X_test, y_test)  
# plt.show()
plt.savefig('static\\svm.png')




model = pickle.load(open("model_dtc.sav", 'rb'))
y_score = model.predict(X_test)
dtacc=accuracy_score(y_test, y_score)
dtacc=round(dtacc*100,2)
print("Dtc :"+ str(dtacc))
plot_confusion_matrix(model, X_test, y_test)  
# plt.show()
plt.savefig('static\\DT.png')

model = pickle.load(open("model_knn.sav", 'rb'))
y_score = model.predict(X_test)
knacc=accuracy_score(y_test, y_score)
knacc=round(knacc*100,2)
print("KNN :"+ str(knacc))
plot_confusion_matrix(model, X_test, y_test)  
# plt.show()
plt.savefig('static\\knn.png')

model = pickle.load(open("model_lr.sav", 'rb'))
y_score = model.predict(X_test)
acc_lr=accuracy_score(y_test, y_score)
acc_lr=round(acc_lr*100,2)
print("LR :"+ str(acc_lr))
plot_confusion_matrix(model, X_test, y_test)  
# plt.show()
plt.savefig('static\\lr.png')


model = pickle.load(open("model_rf.sav", 'rb'))
y_score = model.predict(X_test)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(trainData['Label'])

y_pred = [int(round(x)) for x in y_score]

y_score=list(le.inverse_transform(y_pred))

acc_rf=accuracy_score(y_test, y_score)
acc_rf=round(acc_rf*100,2)
# confusion_matrix(y_test, y_score).savefig('static\\rf.png')

import seaborn as sns
import matplotlib.transforms
annot_kws = {"ha": 'left',"va": 'top'}

cm = confusion_matrix(y_test, y_score)
f = sns.heatmap(cm, annot=False)
for t in f.texts:
    trans = t.get_transform()
    offs = matplotlib.transforms.ScaledTranslation(.5, .5,
                    matplotlib.transforms.IdentityTransform())
    t.set_transform( offs + trans )
figure = f.get_figure()    
figure.savefig('static\\rf.png',annot=True, fmt="d",annot_kws=annot_kws )
print("Rf :"+ str(acc_rf))

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

x=['SVM','DT','KNN','LR','RF']
y=[svnacc,dtacc,knacc,acc_lr,acc_rf]
plt.bar(x, y) 
plt.savefig('static\\compare.png',dpi=300,bbox_inches='tight')



# flask start

#RUN WEBSERVER


# from werkzeug.wrappers import Request, Response
from flask import Flask,redirect,url_for,render_template,request
app = Flask(__name__)

@app.route("/")
def hello():
#     return "Hello World!"
    return render_template("index.html")
@app.route("/dataset")
def dataset():
    return render_template("dataset.html",tables=[dispdata.to_html(classes='data')], titles=trainData.columns.values)
    
@app.route("/preproce")
def features():
    return render_template("preproce.html",tables=[trainData.to_html(classes='data')], titles=trainData.columns.values)

@app.route("/split")
def split():
    return render_template("split.html",dssize=train_vectors.shape[0],trsize=X_train.shape[0],tssize=X_test.shape[0])

@app.route("/classification")
def classification():
    return render_template("classification.html",knacc=knacc,dtacc=dtacc,svnacc=svnacc,acc_rf=acc_rf,acc_lr=acc_lr)

@app.route("/results")
def results():
    return render_template("graph.html")
# 
@app.route("/analyze")
def analyze():
    return render_template("graph1.html")
@app.route("/message", methods=['GET', 'POST'])
def message():
    res=""
    if request.method=="POST":
        msg=request.form["msg"]
        trainData=read_excel('spam.xls') # DATA FRAME -> COL, ROWS
        df2 = {'Label': 'ham', 'Content': 'hello how are you'}
        trainData = trainData.append(df2, ignore_index = True)
        processed_review = []
        for i in trainData["Content"].values:
            sentance = html_tag(i)
            sentance = decontracted(sentance)
            sentance = re.sub("\S*\d\S*", "", sentance)
            sentance = re.sub('[^A-Za-z]+', ' ', sentance)
            sentance = " ".join(i.lower() for i in sentance.split() if i.lower() not in stopwords)
            processed_review.append(sentance)
        trainData["Cleantext"] = processed_review
        vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
        train_vectors = vectorizer.fit_transform(trainData['Cleantext'])
        X_test1=train_vectors[train_vectors.shape[0]-1]
        model = pickle.load(open("model_dtc.sav", 'rb'))
        # y_score = model.predict(X_test)
        y_score = model.predict(X_test1)
        res="This Message is :"+ y_score[0]
    return render_template("testmanual.html",res=res)


# 


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)




100%|████████████████████████████████████████████████████████████████████████████| 5571/5571 [00:05<00:00, 1088.71it/s]


ImportError: cannot import name 'plot_confusion_matrix'

In [2]:
from flask_mysqldb import MySQL
import MySQLdb.cursors

app = Flask(__name__)
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'mask'
mysql = MySQL(app)

#in function

sql = "select * from mask"
print(len(sql))
cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
cursor.execute(sql)
rows = cursor.fetchall()

NameError: name 'Flask' is not defined

In [30]:
from datetime import datetime
tm1=datetime.now()


In [31]:
tm2=tm1
tm1=datetime.now()

In [32]:
tdiff=tm1-tm2

print(tdiff.total_seconds())
if tdiff.total_seconds()>7:
    print("alert")

0.732232


In [29]:
from datetime import date, datetime

today_date = date.today()
# date_time = datetime.strptime(date_time_string, '%Y-%m-%d %H:%M')